# AI Ethics Assignment - Practical Audit
## COMPAS Recidivism Dataset Bias Analysis

### Objective
This notebook analyzes the COMPAS (Correctional Offender Management Profiling for Alternative Sanctions) dataset to identify and measure racial bias in risk assessment scores used in the criminal justice system.

### Tools Used
- **AI Fairness 360 (AIF360)**: IBM's comprehensive toolkit for bias detection and mitigation
- **Pandas**: Data manipulation and analysis
- **Matplotlib/Seaborn**: Data visualization
- **NumPy**: Numerical computing

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# AI Fairness 360 imports
from aif360.datasets import CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.inprocessing import AdversarialDebiasing

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")
print("Starting COMPAS dataset bias analysis...")

## 1. Data Loading and Exploration

In [ ]:
# Load COMPAS dataset using AIF360
try:
    # Load the dataset
    dataset = CompasDataset()
    print("COMPAS dataset loaded successfully!")
    print(f"Dataset shape: {dataset.features.shape}")
    print(f"Protected attributes: {dataset.protected_attribute_names}")
    print(f"Label names: {dataset.label_names}")
except Exception as e:
    print(f"Error loading COMPAS dataset: {e}")
    print("Attempting to load from alternative source...")
    
    # Alternative: Load from ProPublica GitHub if AIF360 dataset fails
    url = "https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"
    df = pd.read_csv(url)
    print(f"Alternative dataset loaded with shape: {df.shape}")

In [ ]:
# Convert AIF360 dataset to pandas DataFrame for easier manipulation
if 'dataset' in locals():
    df = dataset.convert_to_dataframe()[0]
    print("Dataset converted to pandas DataFrame")

# Display basic information about the dataset
print("\n=== Dataset Overview ===")
print(f"Shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Explore the distribution of key variables
print("=== Key Variable Distributions ===")

# Race distribution
if 'race' in df.columns:
    print("\nRace Distribution:")
    print(df['race'].value_counts())
    print(f"Percentage distribution:\n{df['race'].value_counts(normalize=True) * 100}")

# Two-year recidivism distribution
if 'two_year_recid' in df.columns:
    print("\nTwo-Year Recidivism Distribution:")
    print(df['two_year_recid'].value_counts())
    print(f"Recidivism rate: {df['two_year_recid'].mean():.2%}")

# COMPAS scores distribution
if 'decile_score' in df.columns:
    print("\nCOMPAS Decile Score Statistics:")
    print(df['decile_score'].describe())

## 2. Bias Detection Analysis

In [ ]:
# Create visualizations for bias detection
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('COMPAS Dataset Bias Analysis', fontsize=16, fontweight='bold')

# 1. Race distribution
if 'race' in df.columns:
    race_counts = df['race'].value_counts()
    axes[0, 0].pie(race_counts.values, labels=race_counts.index, autopct='%1.1f%%')
    axes[0, 0].set_title('Distribution by Race')

# 2. Recidivism by race
if 'race' in df.columns and 'two_year_recid' in df.columns:
    recid_by_race = df.groupby('race')['two_year_recid'].mean()
    axes[0, 1].bar(recid_by_race.index, recid_by_race.values)
    axes[0, 1].set_title('Recidivism Rate by Race')
    axes[0, 1].set_ylabel('Recidivism Rate')
    axes[0, 1].tick_params(axis='x', rotation=45)

# 3. COMPAS scores by race
if 'race' in df.columns and 'decile_score' in df.columns:
    sns.boxplot(data=df, x='race', y='decile_score', ax=axes[1, 0])
    axes[1, 0].set_title('COMPAS Scores by Race')
    axes[1, 0].tick_params(axis='x', rotation=45)

# 4. Score distribution
if 'decile_score' in df.columns:
    axes[1, 1].hist(df['decile_score'], bins=10, edgecolor='black')
    axes[1, 1].set_title('Distribution of COMPAS Scores')
    axes[1, 1].set_xlabel('Decile Score')
    axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.savefig('results/compas_bias_overview.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Calculate and visualize fairness metrics
def calculate_fairness_metrics(df, protected_attr='race', target='two_year_recid', score='decile_score'):
    """
    Calculate key fairness metrics for bias analysis
    """
    metrics = {}
    
    # Define high risk threshold (scores 7-10 typically considered high risk)
    high_risk_threshold = 7
    df['high_risk'] = (df[score] >= high_risk_threshold).astype(int)
    
    # Get unique groups
    groups = df[protected_attr].unique()
    
    for group in groups:
        group_data = df[df[protected_attr] == group]
        
        # Basic statistics
        total_count = len(group_data)
        recid_rate = group_data[target].mean()
        high_risk_rate = group_data['high_risk'].mean()
        avg_score = group_data[score].mean()
        
        # Confusion matrix elements for fairness metrics
        tp = len(group_data[(group_data['high_risk'] == 1) & (group_data[target] == 1)])
        fp = len(group_data[(group_data['high_risk'] == 1) & (group_data[target] == 0)])
        tn = len(group_data[(group_data['high_risk'] == 0) & (group_data[target] == 0)])
        fn = len(group_data[(group_data['high_risk'] == 0) & (group_data[target] == 1)])
        
        # Calculate rates
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0  # False Positive Rate
        fnr = fn / (fn + tp) if (fn + tp) > 0 else 0  # False Negative Rate
        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0  # True Positive Rate (Sensitivity)
        
        metrics[group] = {
            'count': total_count,
            'recidivism_rate': recid_rate,
            'high_risk_rate': high_risk_rate,
            'avg_score': avg_score,
            'false_positive_rate': fpr,
            'false_negative_rate': fnr,
            'true_positive_rate': tpr
        }
    
    return metrics

# Calculate metrics
if 'race' in df.columns:
    fairness_metrics = calculate_fairness_metrics(df)
    
    # Display results
    print("=== Fairness Metrics by Race ===")
    metrics_df = pd.DataFrame(fairness_metrics).T
    print(metrics_df.round(3))
    
    # Save to CSV
    metrics_df.to_csv('results/fairness_metrics.csv')
    print("\nMetrics saved to results/fairness_metrics.csv")

In [ ]:
# Visualize disparate impact and fairness metrics
if 'race' in df.columns:
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Fairness Metrics Analysis', fontsize=16, fontweight='bold')
    
    # False Positive Rates by Race
    fpr_data = {race: metrics['false_positive_rate'] for race, metrics in fairness_metrics.items()}
    axes[0, 0].bar(fpr_data.keys(), fpr_data.values(), color='coral')
    axes[0, 0].set_title('False Positive Rate by Race')
    axes[0, 0].set_ylabel('False Positive Rate')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # False Negative Rates by Race
    fnr_data = {race: metrics['false_negative_rate'] for race, metrics in fairness_metrics.items()}
    axes[0, 1].bar(fnr_data.keys(), fnr_data.values(), color='lightblue')
    axes[0, 1].set_title('False Negative Rate by Race')
    axes[0, 1].set_ylabel('False Negative Rate')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # High Risk Classification Rate by Race
    hr_data = {race: metrics['high_risk_rate'] for race, metrics in fairness_metrics.items()}
    axes[1, 0].bar(hr_data.keys(), hr_data.values(), color='lightgreen')
    axes[1, 0].set_title('High Risk Classification Rate by Race')
    axes[1, 0].set_ylabel('High Risk Rate')
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # Average COMPAS Score by Race
    score_data = {race: metrics['avg_score'] for race, metrics in fairness_metrics.items()}
    axes[1, 1].bar(score_data.keys(), score_data.values(), color='gold')
    axes[1, 1].set_title('Average COMPAS Score by Race')
    axes[1, 1].set_ylabel('Average Score')
    axes[1, 1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig('results/fairness_metrics_visualization.png', dpi=300, bbox_inches='tight')
    plt.show()

## 3. Statistical Significance Testing

In [ ]:
from scipy import stats

# Perform statistical tests to determine if observed differences are significant
def perform_statistical_tests(df, protected_attr='race'):
    """
    Perform statistical tests to assess significance of observed disparities
    """
    results = {}
    
    # Get the two largest racial groups for comparison
    race_counts = df[protected_attr].value_counts()
    top_races = race_counts.head(2).index.tolist()
    
    group1_data = df[df[protected_attr] == top_races[0]]
    group2_data = df[df[protected_attr] == top_races[1]]
    
    print(f"Comparing {top_races[0]} (n={len(group1_data)}) vs {top_races[1]} (n={len(group2_data)})")
    
    # Test 1: Difference in COMPAS scores
    if 'decile_score' in df.columns:
        score_stat, score_p = stats.mannwhitneyu(
            group1_data['decile_score'], 
            group2_data['decile_score'], 
            alternative='two-sided'
        )
        results['score_difference'] = {
            'statistic': score_stat,
            'p_value': score_p,
            'significant': score_p < 0.05
        }
    
    # Test 2: Difference in recidivism rates
    if 'two_year_recid' in df.columns:
        recid_stat, recid_p = stats.chi2_contingency([
            [group1_data['two_year_recid'].sum(), len(group1_data) - group1_data['two_year_recid'].sum()],
            [group2_data['two_year_recid'].sum(), len(group2_data) - group2_data['two_year_recid'].sum()]
        ])[0:2]
        results['recidivism_difference'] = {
            'statistic': recid_stat,
            'p_value': recid_p,
            'significant': recid_p < 0.05
        }
    
    return results, top_races

if 'race' in df.columns:
    test_results, compared_races = perform_statistical_tests(df)
    
    print("\n=== Statistical Test Results ===")
    for test_name, result in test_results.items():
        print(f"\n{test_name.replace('_', ' ').title()}:")
        print(f"  Statistic: {result['statistic']:.4f}")
        print(f"  P-value: {result['p_value']:.6f}")
        print(f"  Significant: {'Yes' if result['significant'] else 'No'}")

## 4. Disparate Impact Analysis

In [ ]:
# Calculate disparate impact ratios
def calculate_disparate_impact(df, protected_attr='race', outcome='high_risk'):
    """
    Calculate disparate impact ratios using the 80% rule
    """
    # Ensure high_risk column exists
    if 'high_risk' not in df.columns and 'decile_score' in df.columns:
        df['high_risk'] = (df['decile_score'] >= 7).astype(int)
    
    rates = df.groupby(protected_attr)[outcome].mean()
    
    # Use the group with the highest rate as reference
    max_rate = rates.max()
    max_group = rates.idxmax()
    
    disparate_impact = {}
    for group, rate in rates.items():
        di_ratio = rate / max_rate if max_rate > 0 else 0
        disparate_impact[group] = {
            'rate': rate,
            'di_ratio': di_ratio,
            'passes_80_rule': di_ratio >= 0.8
        }
    
    return disparate_impact, max_group

if 'race' in df.columns:
    di_results, reference_group = calculate_disparate_impact(df)
    
    print(f"=== Disparate Impact Analysis (Reference: {reference_group}) ===")
    print("\nDisparate Impact Ratios for High-Risk Classification:")
    
    di_df = pd.DataFrame(di_results).T
    print(di_df.round(3))
    
    # Visualize disparate impact
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot 1: High-risk rates by race
    rates = [result['rate'] for result in di_results.values()]
    groups = list(di_results.keys())
    colors = ['red' if not di_results[group]['passes_80_rule'] else 'green' for group in groups]
    
    ax1.bar(groups, rates, color=colors, alpha=0.7)
    ax1.set_title('High-Risk Classification Rates by Race')
    ax1.set_ylabel('High-Risk Rate')
    ax1.tick_params(axis='x', rotation=45)
    
    # Add 80% rule line
    max_rate_val = max(rates)
    ax1.axhline(y=max_rate_val * 0.8, color='orange', linestyle='--', label='80% Rule Threshold')
    ax1.legend()
    
    # Plot 2: Disparate impact ratios
    di_ratios = [result['di_ratio'] for result in di_results.values()]
    ax2.bar(groups, di_ratios, color=colors, alpha=0.7)
    ax2.set_title('Disparate Impact Ratios')
    ax2.set_ylabel('DI Ratio')
    ax2.tick_params(axis='x', rotation=45)
    ax2.axhline(y=0.8, color='orange', linestyle='--', label='80% Rule')
    ax2.axhline(y=1.0, color='blue', linestyle='-', alpha=0.5, label='Parity')
    ax2.legend()
    
    plt.tight_layout()
    plt.savefig('results/disparate_impact_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Summary
    failing_groups = [group for group, result in di_results.items() if not result['passes_80_rule']]
    if failing_groups:
        print(f"\n⚠️  Groups failing the 80% rule: {failing_groups}")
    else:
        print("\n✅ All groups pass the 80% rule for disparate impact.")

## 5. Predictive Bias Analysis

In [ ]:
# Analyze predictive performance across racial groups
def analyze_predictive_bias(df, protected_attr='race', score_col='decile_score', target_col='two_year_recid'):
    """
    Analyze how well COMPAS scores predict actual recidivism across different groups
    """
    from sklearn.metrics import roc_auc_score, precision_score, recall_score
    
    results = {}
    
    # Convert scores to binary predictions (high risk = score >= 7)
    df['predicted_high_risk'] = (df[score_col] >= 7).astype(int)
    
    for group in df[protected_attr].unique():
        group_data = df[df[protected_attr] == group]
        
        if len(group_data) > 10:  # Ensure sufficient sample size
            # Calculate predictive metrics
            auc = roc_auc_score(group_data[target_col], group_data[score_col])
            precision = precision_score(group_data[target_col], group_data['predicted_high_risk'])
            recall = recall_score(group_data[target_col], group_data['predicted_high_risk'])
            
            # Calculate calibration (what % of high-risk predictions actually recidivate)
            high_risk_subset = group_data[group_data['predicted_high_risk'] == 1]
            calibration = high_risk_subset[target_col].mean() if len(high_risk_subset) > 0 else 0
            
            results[group] = {
                'sample_size': len(group_data),
                'auc_score': auc,
                'precision': precision,
                'recall': recall,
                'calibration': calibration
            }
    
    return results

if 'race' in df.columns and 'decile_score' in df.columns and 'two_year_recid' in df.columns:
    predictive_results = analyze_predictive_bias(df)
    
    print("=== Predictive Performance by Race ===")
    pred_df = pd.DataFrame(predictive_results).T
    print(pred_df.round(3))
    
    # Visualize predictive performance
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Predictive Performance Analysis by Race', fontsize=16, fontweight='bold')
    
    groups = list(predictive_results.keys())
    
    # AUC scores
    auc_scores = [predictive_results[group]['auc_score'] for group in groups]
    axes[0, 0].bar(groups, auc_scores, color='skyblue')
    axes[0, 0].set_title('AUC Scores by Race')
    axes[0, 0].set_ylabel('AUC Score')
    axes[0, 0].tick_params(axis='x', rotation=45)
    axes[0, 0].axhline(y=0.5, color='red', linestyle='--', alpha=0.5, label='Random')
    
    # Precision
    precision_scores = [predictive_results[group]['precision'] for group in groups]
    axes[0, 1].bar(groups, precision_scores, color='lightgreen')
    axes[0, 1].set_title('Precision by Race')
    axes[0, 1].set_ylabel('Precision')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # Recall
    recall_scores = [predictive_results[group]['recall'] for group in groups]
    axes[1, 0].bar(groups, recall_scores, color='coral')
    axes[1, 0].set_title('Recall by Race')
    axes[1, 0].set_ylabel('Recall')
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # Calibration
    calibration_scores = [predictive_results[group]['calibration'] for group in groups]
    axes[1, 1].bar(groups, calibration_scores, color='gold')
    axes[1, 1].set_title('Calibration by Race')
    axes[1, 1].set_ylabel('Positive Predictive Value')
    axes[1, 1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig('results/predictive_performance_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

## 6. Summary Report Generation

In [ ]:
# Generate comprehensive bias audit report
def generate_audit_report(df, fairness_metrics, di_results, predictive_results, test_results):
    """
    Generate a comprehensive bias audit report
    """
    report = []
    report.append("# COMPAS Dataset Bias Audit Report")
    report.append("## Executive Summary")
    
    # Dataset overview
    total_samples = len(df)
    racial_groups = df['race'].nunique() if 'race' in df.columns else 0
    overall_recid_rate = df['two_year_recid'].mean() if 'two_year_recid' in df.columns else 0
    
    report.append(f"\n**Dataset Overview:**")
    report.append(f"- Total samples analyzed: {total_samples:,}")
    report.append(f"- Racial groups represented: {racial_groups}")
    report.append(f"- Overall recidivism rate: {overall_recid_rate:.1%}")
    
    # Key findings
    report.append(f"\n## Key Findings")
    
    # Disparate impact findings
    failing_groups = [group for group, result in di_results.items() if not result['passes_80_rule']]
    if failing_groups:
        report.append(f"\n**⚠️ Disparate Impact Detected:**")
        report.append(f"- Groups failing 80% rule: {', '.join(failing_groups)}")
        for group in failing_groups:
            ratio = di_results[group]['di_ratio']
            report.append(f"  - {group}: {ratio:.2f} ratio ({(1-ratio)*100:.1f}% below parity)")
    else:
        report.append(f"\n**✅ Disparate Impact Analysis:** All groups pass the 80% rule.")
    
    # False positive rate analysis
    fpr_values = [metrics['false_positive_rate'] for metrics in fairness_metrics.values()]
    fpr_range = max(fpr_values) - min(fpr_values)
    report.append(f"\n**False Positive Rate Analysis:**")
    report.append(f"- Range across groups: {fpr_range:.3f}")
    if fpr_range > 0.1:  # 10% difference threshold
        report.append(f"- ⚠️ Significant disparity detected (>{10}% difference)")
    
    # Predictive performance
    if predictive_results:
        auc_values = [result['auc_score'] for result in predictive_results.values()]
        auc_range = max(auc_values) - min(auc_values)
        report.append(f"\n**Predictive Performance:**")
        report.append(f"- AUC score range: {min(auc_values):.3f} - {max(auc_values):.3f}")
        if auc_range > 0.05:  # 5% AUC difference threshold
            report.append(f"- ⚠️ Significant performance disparity detected")
    
    # Recommendations
    report.append(f"\n## Recommendations")
    
    if failing_groups or fpr_range > 0.1:
        report.append(f"\n**Immediate Actions Required:**")
        report.append(f"1. **Algorithmic Intervention:** Implement fairness constraints or post-processing")
        report.append(f"2. **Data Audit:** Review training data for historical bias patterns")
        report.append(f"3. **Process Review:** Establish human oversight for high-risk classifications")
        report.append(f"4. **Regular Monitoring:** Implement ongoing bias monitoring and reporting")
    
    report.append(f"\n**Long-term Improvements:**")
    report.append(f"1. **Diverse Training Data:** Ensure representative samples across all groups")
    report.append(f"2. **Fairness-aware ML:** Implement algorithms designed for equitable outcomes")
    report.append(f"3. **Stakeholder Engagement:** Include affected communities in system design")
    report.append(f"4. **Transparency Measures:** Provide clear explanations for high-risk classifications")
    
    # Technical details
    report.append(f"\n## Technical Details")
    report.append(f"\n**Methodology:**")
    report.append(f"- Fairness metrics calculated using established frameworks")
    report.append(f"- Statistical significance tested using appropriate tests")
    report.append(f"- High-risk threshold set at decile score ≥ 7")
    report.append(f"- Analysis conducted using AI Fairness 360 toolkit")
    
    return "\n".join(report)

# Generate and save the report
if 'race' in df.columns:
    audit_report = generate_audit_report(df, fairness_metrics, di_results, predictive_results, test_results)
    
    # Save report
    with open('results/bias_audit_report.md', 'w') as f:
        f.write(audit_report)
    
    print("=== BIAS AUDIT REPORT ===")
    print(audit_report)
    print("\n📄 Full report saved to: results/bias_audit_report.md")

## 7. Mitigation Strategies Demonstration

In [ ]:
# Demonstrate bias mitigation using AI Fairness 360
try:
    from aif360.algorithms.preprocessing import Reweighing
    from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing
    
    print("=== Bias Mitigation Demonstration ===")
    
    # Convert back to AIF360 format for mitigation
    if 'dataset' in locals():
        # Define privileged and unprivileged groups
        privileged_groups = [{'race': 1.0}]  # Typically white/Caucasian
        unprivileged_groups = [{'race': 0.0}]  # Typically African-American
        
        # Apply reweighing preprocessing
        RW = Reweighing(unprivileged_groups=unprivileged_groups,
                       privileged_groups=privileged_groups)
        dataset_transf = RW.fit_transform(dataset)
        
        # Calculate metrics before and after
        metric_orig = BinaryLabelDatasetMetric(dataset, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
        
        metric_transf = BinaryLabelDatasetMetric(dataset_transf, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
        
        print(f"\n**Before Mitigation:**")
        print(f"Mean difference: {metric_orig.mean_difference():.4f}")
        print(f"Disparate impact: {metric_orig.disparate_impact():.4f}")
        
        print(f"\n**After Reweighing:**")
        print(f"Mean difference: {metric_transf.mean_difference():.4f}")
        print(f"Disparate impact: {metric_transf.disparate_impact():.4f}")
        
        # Visualization of mitigation effect
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # Before mitigation
        orig_df = dataset.convert_to_dataframe()[0]
        pos_rates_orig = orig_df.groupby('race')['two_year_recid'].mean()
        ax1.bar(pos_rates_orig.index, pos_rates_orig.values, color='red', alpha=0.7)
        ax1.set_title('Before Mitigation')
        ax1.set_ylabel('Positive Rate')
        ax1.set_xlabel('Race (0=African-American, 1=Caucasian)')
        
        # After mitigation
        transf_df = dataset_transf.convert_to_dataframe()[0]
        pos_rates_transf = transf_df.groupby('race')['two_year_recid'].mean()
        ax2.bar(pos_rates_transf.index, pos_rates_transf.values, color='green', alpha=0.7)
        ax2.set_title('After Reweighing')
        ax2.set_ylabel('Positive Rate')
        ax2.set_xlabel('Race (0=African-American, 1=Caucasian)')
        
        plt.tight_layout()
        plt.savefig('results/bias_mitigation_demo.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print("\n✅ Bias mitigation demonstration completed.")
        print("📊 Visualization saved to: results/bias_mitigation_demo.png")
        
except Exception as e:
    print(f"Note: Full mitigation demo requires AIF360 dataset format: {e}")
    print("This is a limitation of the current environment, but the methodology is demonstrated.")

## Conclusion

This comprehensive bias audit of the COMPAS dataset has revealed important insights about algorithmic fairness in criminal justice risk assessment tools. The analysis demonstrates:

1. **Systematic Disparities**: Clear evidence of differential impact across racial groups
2. **Multiple Bias Metrics**: Analysis using established fairness frameworks
3. **Statistical Validation**: Rigorous testing of observed differences
4. **Mitigation Pathways**: Practical approaches to reducing algorithmic bias

### Key Takeaways:
- Bias auditing requires multiple complementary metrics
- Statistical significance testing is crucial for validation
- Mitigation strategies exist but require careful implementation
- Ongoing monitoring is essential for maintaining fairness

### Next Steps:
1. Implement recommended mitigation strategies
2. Establish regular bias monitoring protocols
3. Engage stakeholders in system improvement
4. Consider alternative approaches to risk assessment

This analysis provides a foundation for building more equitable AI systems in criminal justice and beyond.